In [1]:
# Version	            Python version	   Compiler	    Build tools	  cuDNN	  CUDA
# tensorflow_gpu-2.10.0	   3.7-3.10	       MSVC 2019	Bazel 5.1.1	   8.1	  11.2

# %pip install nvidia-pyindex
# %pip install tensorflow-gpu==2.10.0
# %pip install torch
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# %pip install transformers
# %pip install transformers[torch]
# %pip install accelerate>=0.26.0

Preprocessed Dataset (for Bert Model Training)

1. Splitting the Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your dataset
data = pd.read_csv("datasets/sentence_data.csv")  # Ensure your dataset is in the correct CSV format

# Split the dataset
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

# Save the splits for future use
train_data.to_csv("datasets/train_data.csv", index=False)
val_data.to_csv("datasets/val_data.csv", index=False)
test_data.to_csv("datasets/test_data.csv", index=False)

print(f"Training set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


In [ ]:
print(train_data.columns)
print(val_data.columns)

Phase 1: Spell Correction

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")
model = BertForSequenceClassification.from_pretrained("sagorsarker/bangla-bert-base", num_labels=2)

# Tokenize data
train_encodings = tokenizer(list(train_data['synth_spelling_errors']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_data['synth_spelling_errors']), truncation=True, padding=True)

# Convert to PyTorch dataset
import torch
class BanglaDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

train_labels = list(train_data['cleaned_text'])
val_labels = list(val_data['cleaned_text'])

train_dataset = BanglaDataset(train_encodings, train_labels)
val_dataset = BanglaDataset(val_encodings, val_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model
trainer.train()

Phase 2: Grammar Correction

In [ ]:
# Install GECToR dependencies (done once)
# !pip install allennlp allennlp-models

from allennlp.predictors import Predictor
from allennlp_models.pretrained import load_predictor

# Load GECToR pre-trained model
predictor = load_predictor("https://storage.googleapis.com/allennlp-public-models/roberta-base-2020.06.09.tar.gz")

# Example prediction
example_sentence = "তোমরা ভুলগুলো চিহ্নিত করনি।"
prediction = predictor.predict(sentence=example_sentence)

# Output
print("Original:", example_sentence)
print("Corrected:", prediction["tokens"])


Phase 3: Sentence Correction

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load T5 model and tokenizer
model_name = "google/mt5-small"  # Use T5 pre-trained model for multilingual tasks
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize data for T5
def preprocess_function(examples):
    inputs = [f"grammar: {i}" for i in examples["incorrect_sentence"]]
    outputs = examples["cleaned_text"]
    model_inputs = tokenizer(inputs, text_target=outputs, max_length=128, truncation=True)
    return model_inputs

# Tokenize train and validation sets
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Fine-tune the model
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()
trainer.save_model("t5_sentence_correction_model")
